In [3]:
old_prompt = """You are given a Czech court decision (“rozhodnutí Nejvyššího správního soudu”):

[DOCUMENT CONTENTS START]
{DOCUMENT_CONTENTS}
[DOCUMENT CONTENTS END]

Your task is to analyze the provided document and generate multiple question–answer pairs about it. The document text is in Czech, but the instructions here are in English.

**Requirements for your output:**
1. **Language:** The questions and answers you produce must be **in Czech**.
2. **Number of QA pairs:** Please produce 5–10 question–answer pairs (depending on the document’s length and complexity).
3. **Question categories:** Each question must correspond to exactly one of these categories (use whichever are relevant, but try to diversify the final set):
   - **Factual (factual)**: Basic factual details, e.g. key dates, numbers, direct quotations.
   - **Reasoning about legal argumentation (reasoning)**: How the court reasoned, references to specific paragraphs, justifications for a particular conclusion.
   - **Interpretation of law (legal_interpretation)**: Questions focusing on how the court applied or interpreted specific statutes, directives, or legal principles.
   - **Credibility / witness or party consistency (credibility)**: Questions about contradictory statements, analysis of witness credibility, or factual inconsistencies identified by the court.
   - **Process aspects (procedure)**: Chronology of steps, who decided what, whether it was an appeal, cassation complaint, etc.
   - **Summary or context (summary)**: A broader summary question about the core issues or overarching legal context.
   - **Practical consequences (practical)**: Questions focusing on the real-world effects of the ruling or instructions for further proceedings.
4. **Structure:** Output a **JSON object** with a top-level key `"qa_pairs"`, containing an **array** of objects. Each object must have these fields:
   - `"question"`: A single question in Czech.
   - `"golden_answer"`: A concise and precise answer in Czech, reflecting the correct or ideal response based on the text.
   - `"questions_type"`: One of the categories above (factual, reasoning, legal_interpretation, credibility, procedure, summary, practical).

**Example JSON output format** (do not produce ellipses "…" in the final output; this is only illustrative):
```
{{
  "qa_pairs": [
    {{
      "question": "V jakém roce podal stěžovatel první žalobu k Městskému soudu v Praze?",
      "golden_answer": "Podal ji v roce 2021.",
      "questions_type": "factual"
    }},
    {{
      "question": "Proč soud odmítl prolomit § 75 odst. 1 s. ř. s. v tomto řízení?",
      "golden_answer": "Protože stěžovatel stále mohl podat novou žádost o azyl, a tím byla zajištěna ochrana non-refoulement.",
      "questions_type": "reasoning"
    }}
  ]
}}

```

**Important**:
- Stay strictly within this JSON structure.
- **Do not** include any additional commentary, markdown formatting, or explanations outside of the JSON.
- Ensure each `"golden_answer"` is factually correct and concise, pulled from or supported by the text.

**Now**:
1. Read carefully the entire Czech court decision above (`{DOCUMENT_CONTENTS}`).
2. Produce a set of **5–10 question–answer pairs** in Czech.
3. Output only the JSON with the specified structure and fields.

Begin.
"""

In [4]:
old_prompt_2 = """
You are given a Czech court decision (“rozhodnutí Nejvyššího správního soudu”):

[DOCUMENT CONTENTS START]
{DOCUMENT_CONTENTS}
[DOCUMENT CONTENTS END]

Your task is to analyze the provided document and generate multiple question–answer pairs about it. The document text is in Czech, but the instructions here are in English.

**Important context:**
- We have thousands of such decisions in a large corpus.
- Each question must reference at least one **unique or specific detail** from the text (such as the spisová značka, date of the decision, or distinctive phrases or facts) so that we can retrieve the correct document among many.

**Requirements for your output:**
1. **Language:** The questions and answers you produce must be **in Czech**.
2. **Number of QA pairs:** Please produce 5–10 question–answer pairs (depending on the document’s length and complexity).
3. **Question categories:** Each question must correspond to exactly one of these categories (use whichever are relevant, but try to diversify the final set):
   - **Factual (factual)**: Basic factual details, e.g. key dates, numbers, direct quotations.
   - **Reasoning about legal argumentation (reasoning)**: How the court reasoned, references to specific paragraphs, justifications for a particular conclusion.
   - **Interpretation of law (legal_interpretation)**: Questions focusing on how the court applied or interpreted specific statutes, directives, or legal principles.
   - **Credibility / witness or party consistency (credibility)**: Questions about contradictory statements, analysis of witness credibility, or factual inconsistencies identified by the court.
   - **Process aspects (procedure)**: Chronology of steps, who decided what, whether it was an appeal, cassation complaint, etc.
   - **Summary or context (summary)**: A broader summary question about the core issues or overarching legal context.
   - **Practical consequences (practical)**: Questions focusing on the real-world effects of the ruling or instructions for further proceedings.
4. **Structure:** Output a **JSON object** with a top-level key `"qa_pairs"`, containing an **array** of objects. Each object must have these fields:
   - `"question"`: A single question in Czech. **Incorporate at least one unique detail from the decision** (e.g., the specific date, spisová značka, or other distinctive text) so that it is clear you are referencing *this* document.
   - `"golden_answer"`: A concise and precise answer in Czech, reflecting the correct or ideal response based on the text.
   - `"questions_type"`: One of the categories above (factual, reasoning, legal_interpretation, credibility, procedure, summary, practical).

**Example JSON output format** (for illustration only):

{{ "qa_pairs": [ {{ "question": "V jakém roce podal stěžovatel první žalobu k Městskému soudu v Praze v rozhodnutí ze dne 10. 5. 2022, sp. zn. 1 Azs 123/2022-45?", "golden_answer": "Podal ji v roce 2021.", "questions_type": "factual" }}, {{ "question": "Proč soud ve spisu 1 Azs 123/2022-45 z 10. 5. 2022 odmítl prolomit § 75 odst. 1 s. ř. s. v tomto řízení?", "golden_answer": "Protože stěžovatel stále mohl podat novou žádost o azyl, a tím byla zajištěna ochrana non-refoulement.", "questions_type": "reasoning" }} ] }}



**Important**:
- Stay strictly within this JSON structure.
- **Do not** include any additional commentary, markdown formatting, or explanations outside of the JSON.
- Each question must clearly reference a **unique detail** from the text.
- Ensure each `"golden_answer"` is factually correct and concise, pulled from or supported by the text.

**Now**:
1. Read carefully the entire Czech court decision above (between [DOCUMENT CONTENTS START] and [DOCUMENT CONTENTS END]).
2. Produce a set of **5–10 question–answer pairs** in Czech, referencing unique details so we can identify the correct decision among many.
3. Output only the JSON with the specified structure and fields.

Begin.
"""

In [47]:
prompt = """
You are given a Czech court decision (“rozhodnutí Nejvyššího správního soudu”):

[DOCUMENT CONTENTS START]
{DOCUMENT_CONTENTS}
[DOCUMENT CONTENTS END]

Your task is to:
1. Pretend this decision does not yet exist.
2. Formulate one question in Czech that a lawyer might ask regarding a real-world scenario that parallels or anticipates the content of this (as-yet-nonexistent) court decision. This question may concern matters such as the final verdict, the possibility of appealing the outcome, whether one party can claim compensation of legal expenses, or any other issue arising from the hypothetical scenario.
3. Provide a concise answer—also in Czech—that an LLM RAG system might give, assuming it has access to all historical „rozhodnutí Nejvyššího správního soudu“ and is trying to predict how such a case could be decided under Czech law.

**Output Format Requirements:**
- Return your result as a **single JSON object** with the following top-level keys:
- That element must be an object with the following keys:
  - `"question"`: The single Czech-language question posed from a lawyer’s perspective (as if the court decision does not exist yet).
  - `"golden_answer"`: A short, model-like answer (in Czech) describing how Czech courts might resolve such a scenario.
  - `"questions_type"`: For consistency with our broader system, set this value to a relevant category (e.g., `"reasoning"`, `"legal_interpretation"`, etc.).

Your final output must look like this (example structure only):

```
{{ "question": "…", "golden_answer": "…", "questions_type": "…" }}
```

**Important**:
- Do NOT add any additional commentary or formatting outside of this JSON.
- The `"question"` must be in Czech and reflect a realistic inquiry a lawyer might have about a potential court dispute (similar to what is described in the provided text).
- The `"golden_answer"` must also be in Czech, be relatively concise, and capture how such a scenario could be decided by Czech courts, drawing on similar historical precedents or legal principles.
- Use whichever `"questions_type"` fits best from our established categories (for example: `"factual"`, `"reasoning"`, `"legal_interpretation"`, `"credibility"`, `"procedure"`, `"summary"`, or `"practical"`).

Begin.
"""

In [48]:
# ! pip install langchain-google-genai

In [49]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [50]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    # model="gemini-2.5-pro-exp-03-25",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [51]:
documents = os.listdir("../files")

In [52]:
documents = [doc for doc in documents if doc.endswith(".txt")]

In [53]:
questions = {}

In [54]:
# filter only documents newever than 2020
documents = [doc for doc in documents if int(doc.split('.')[0].split("_")[-1]) > 2020]
len(documents)

564

In [56]:
# get random 100 documents
import random
documents_subset = random.sample(documents, 200)

In [57]:
documents_subset[:1]

['4607_2024.txt']

In [60]:

questions

{'4607_2024.txt': AIMessage(content='```json\n{\n  "question": "Pokud finanční úřad zadržuje nadměrný odpočet DPH po dobu daňové kontroly, je úroková sazba stanovená jako repo sazba ČNB zvýšená o 2 procentní body v souladu s právem EU, i když daňový subjekt tvrdí, že skutečné náklady na překlenovací úvěr byly vyšší?",\n  "golden_answer": "Podle judikatury Nejvyššího správního soudu, konkrétně rozsudku 8 Afs 274/2022, je úroková sazba ve výši repo sazby ČNB zvýšené o 2 procentní body v souladu s právem EU, pokud pokrývá průměrné náklady na úvěry poskytované nefinančním podnikům. Individuální nároky na náhradu škody převyšující stanovený úrok je třeba uplatnit samostatně, ale samotná sazba je považována za vyhovující požadavkům EU.",\n  "questions_type": "legal_interpretation"\n}\n```', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-a8c25a2d-e3

In [61]:
from time import sleep
from tqdm import tqdm

for doc in tqdm(documents_subset[16:]):
    with open(os.path.join("../files", doc), "r", encoding="utf-8") as f:
        text = f.read()
    prompt_text = prompt.format(DOCUMENT_CONTENTS=text)
    response = llm.invoke(prompt_text)
    questions[doc] = response
    sleep(4)

100%|██████████| 184/184 [18:20<00:00,  5.98s/it]


In [62]:
import json

txt = str(response.content)
try:
    json_response = json.loads(txt)
except json.decoder.JSONDecodeError:
    if "```json" in txt:
        txt = txt.split("```json")[1].split("```")[0]
        json_response = json.loads(txt)
    else:
        raise ValueError(f"Could not parse JSON response: {txt}")
json_response

{'question': 'Klient pobírá invalidní důchod z mládí ze Slovenska přiznaný ještě za Československa. Nyní pracuje v České republice a platí zde sociální pojištění. Může si nárokovat přiznání invalidního důchodu i v České republice, aby se mu zohlednila doba pojištění získaná po rozdělení Československa?',
 'golden_answer': 'Ne, podle ustálené judikatury Nejvyššího správního soudu a Smlouvy mezi Českou republikou a Slovenskou republikou o sociálním zabezpečení (čl. 33) nemá poživatel slovenského invalidního důchodu přiznaného před rozdělením ČSFR nárok na souběžný invalidní důchod v České republice, i když splňuje podmínky pro jeho přiznání, neboť by došlo k dvojímu započtení doby pojištění.',
 'questions_type': 'legal_interpretation'}

In [63]:
extracted_questions = []
for file, response in questions.items():
    try:
        json_response = json.loads(str(response.content))
        json_response['file'] = file
    except json.decoder.JSONDecodeError:
        if "```json" in str(response.content):
            txt = str(response.content).split("```json")[1].split("```")[0]
            json_response = json.loads(txt)
            json_response['file'] = file
        else:
            raise ValueError(f"Could not parse JSON response: {response.content}")
    extracted_questions.append(json_response)

In [64]:
extracted_questions

[{'question': 'Pokud finanční úřad zadržuje nadměrný odpočet DPH po dobu daňové kontroly, je úroková sazba stanovená jako repo sazba ČNB zvýšená o 2 procentní body v souladu s právem EU, i když daňový subjekt tvrdí, že skutečné náklady na překlenovací úvěr byly vyšší?',
  'golden_answer': 'Podle judikatury Nejvyššího správního soudu, konkrétně rozsudku 8 Afs 274/2022, je úroková sazba ve výši repo sazby ČNB zvýšené o 2 procentní body v souladu s právem EU, pokud pokrývá průměrné náklady na úvěry poskytované nefinančním podnikům. Individuální nároky na náhradu škody převyšující stanovený úrok je třeba uplatnit samostatně, ale samotná sazba je považována za vyhovující požadavkům EU.',
  'questions_type': 'legal_interpretation',
  'file': '4607_2024.txt'},
 {'question': 'Pokud zpravodajská služba odmítne soudu poskytnout bližší informace o zdrojích utajované informace, na základě které správní orgán zrušil cizinci trvalý pobyt, protože by to ohrozilo její činnost, může soud toto rozhodnut

In [65]:
formatted_questions = []
global_counter = 0
for response in extracted_questions:
    formatted_questions.append({
            "id": global_counter,
            "question": response["question"],
            "golden_answers": [response["golden_answer"]],
            "metadata": {
                "questions_type": response["questions_type"],
                "file": response["file"],
            }
        })
    # for qa_pair in response["qa_pairs"]:
    #
    global_counter += 1


In [66]:
len(formatted_questions)


200

In [67]:

formatted_questions

[{'id': 0,
  'question': 'Pokud finanční úřad zadržuje nadměrný odpočet DPH po dobu daňové kontroly, je úroková sazba stanovená jako repo sazba ČNB zvýšená o 2 procentní body v souladu s právem EU, i když daňový subjekt tvrdí, že skutečné náklady na překlenovací úvěr byly vyšší?',
  'golden_answers': ['Podle judikatury Nejvyššího správního soudu, konkrétně rozsudku 8 Afs 274/2022, je úroková sazba ve výši repo sazby ČNB zvýšené o 2 procentní body v souladu s právem EU, pokud pokrývá průměrné náklady na úvěry poskytované nefinančním podnikům. Individuální nároky na náhradu škody převyšující stanovený úrok je třeba uplatnit samostatně, ale samotná sazba je považována za vyhovující požadavkům EU.'],
  'metadata': {'questions_type': 'legal_interpretation',
   'file': '4607_2024.txt'}},
 {'id': 1,
  'question': 'Pokud zpravodajská služba odmítne soudu poskytnout bližší informace o zdrojích utajované informace, na základě které správní orgán zrušil cizinci trvalý pobyt, protože by to ohrozil

In [68]:
with open("../nss_questions_v2_full.jsonl", "w", encoding="utf-8") as f:
    for question in formatted_questions:
        f.write(json.dumps(question) + "\n")

In [62]:
questions_subset = random.sample(formatted_questions, 100)

In [63]:
with open("nss_questions_v2_100_subset.jsonl", "w", encoding="utf-8") as f:
    for question in questions_subset:
        f.write(json.dumps(question) + "\n")